In [1]:
from sortedcontainers import SortedList
from scipy.stats import norm
from math import sqrt
from scipy.stats import mannwhitneyu
from numpy.random import uniform,poisson,seed,normal

In [2]:
def add(X,Ux,Y,Uy,T,val) :
    lb = X.bisect_left(val)
    rb = X.bisect_right(val)
    t = rb - lb
    lb = Y.bisect_left(val)
    rb = Y.bisect_right(val)
    t = t + rb - lb
    T = T + 3*(t*t + t)   
    if Ux == None :
        Ux = 0.0
    if lb != rb :
        Ux += 0.5*(rb - lb)
    Ux += lb
    X.add(val)  
    if len(Y) > 0 :
        Uy = len(X)*len(Y) - Ux
    return (X,Ux,Y,Uy,T)

In [3]:
def remove(X,Ux,Y,Uy,T,val) :
    X.remove(val)
    lb = X.bisect_left(val)
    rb = X.bisect_right(val)
    t = rb - lb
    lb = Y.bisect_left(val)
    rb = Y.bisect_right(val)
    t = t + rb - lb
    T = T - 3*(t*t + t)  
    if len(X) == 0 :
         Ux = None
    else :
        if lb != rb :
            Ux -= 0.5*(rb - lb)
        Ux -= lb
        if len(Y) > 0 :
            Uy = len(X)*len(Y) - Ux
    return (X,Ux,Y,Uy,T)

In [4]:
class seq_mann_whitney_U :
    def __init__(self) :
        self.X = SortedList()
        self.Y = SortedList()
        self.Ux = None
        self.Uy = None
        self.T = 0.0
    def add_x(self,val) :
        self.X,self.Ux,self.Y,self.Uy,self.T = add(self.X,self.Ux,self.Y,self.Uy,self.T,val)
    def add_y(self,val) :
        self.Y,self.Uy,self.X,self.Ux,self.T = add(self.Y,self.Uy,self.X,self.Ux,self.T,val)
        return self 
    def remove_x(self,val) :
        self.X,self.Ux,self.Y,self.Uy,self.T = remove(self.X,self.Ux,self.Y,self.Uy,self.T,val)
    def remove_y(self,val) :
        self.Y,self.Uy,self.X,self.Ux,self.T = remove(self.Y,self.Uy,self.X,self.Ux,self.T,val)
    def asymptotic_z(self) :
        if self.Ux == None or self.Uy == None :
            return None
        nx = len(self.X)
        ny = len(self.Y)
        n = nx + ny
        mu = nx*ny/2
        U = self.Ux
        if self.Uy > U :
            U = self.Uy  
        sigma = sqrt((mu/6)*(n+1-self.T/(n*(n-1))))
        if sigma == 0.0 :
            return 0.0
        return (U - mu)/sigma

In [5]:
S = seq_mann_whitney_U()

In [6]:
Y = [19,22,16,29,24]
X = [20,11,17,12]

for y in Y :
    S.add_y(y)
for x in X :
    S.add_x(x)

In [7]:
print(S.Ux)
print(S.Uy)
print(S.T)
print(S.asymptotic_z())
print(2.0*norm.sf(S.asymptotic_z()))

3.0
17.0
0.0
1.7146428199482247
0.0864107329737


In [8]:
mannwhitneyu(Y,X,method="asymptotic",use_continuity=False)

MannwhitneyuResult(statistic=17.0, pvalue=0.0864107329737)

In [9]:
seed(1)

In [10]:
X = uniform(0,1,100)
Y = uniform(0,1,100)

In [11]:
S = seq_mann_whitney_U()
for y in Y :
    S.add_y(y)
for x in X :
    S.add_x(x)

In [12]:
print(S.Ux)
print(S.Uy)
print(S.T)
print(S.asymptotic_z())
print(2.0*norm.sf(S.asymptotic_z()))

5024.0
4976.0
0.0
0.058641333291026505
0.9532377881057771


In [13]:
mannwhitneyu(X,Y,method="asymptotic",use_continuity=False)

MannwhitneyuResult(statistic=5024.0, pvalue=0.9532377881057771)

In [14]:
X = list(X) + [0.5,0.5]
Y = list(Y) + [0.5,0.5]

In [15]:
S = seq_mann_whitney_U()
for y in Y :
    S.add_y(y)
for x in X :
    S.add_x(x)

In [16]:
print(S.Ux)
print(S.Uy)
print(S.T)
print(S.asymptotic_z())
print(2.0*norm.sf(S.asymptotic_z()))

5216.0
5188.0
60.0
0.033208028349460524
0.973508695957846


In [17]:
mannwhitneyu(X,Y,method="asymptotic",use_continuity=False)

MannwhitneyuResult(statistic=5216.0, pvalue=0.973508695957846)

### online change detection

In [18]:
class ocd :
    def __init__(self,n,m) :
        self.test = seq_mann_whitney_U()
        self.X = list()
        self.Y = list()
        self.n = n
        self.m = m
        self.z = None
    def push(self,x) :
        if len(self.X) == self.n :
            self.test.remove_x(self.X.pop(0))
        if len(self.Y) == self.m :
            self.test.remove_y(self.Y.pop(0))
        self.X.append(x)
        self.Y.append(x)
        self.test.add_x(x)
        self.test.add_y(x)
        self.z = self.test.asymptotic_z()
        

In [19]:
n = 2000
m = 1000
S = ocd(n,m)
X = list(poisson(1,50000)) + list(poisson(2,50000))
c = 0
for x in X :
    S.push(x)
    c = c +1 
    if S.z > 2.3 :
        print("alarm !!")
        print("detected at ",c)
        break

alarm !!
detected at  50210
